<a href="https://colab.research.google.com/github/evegat/usm2025-1-dip-ia-edu-v2-evegat/blob/main/InformeFinalDIPLOIAUSMEVT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dimensiones contextuales que promueven el interés por estudiar pedagogía en los egresados de cuarto medio  
## Detector de intereses para la Región Metropolitana  

---

## 1. Introducción

La disminución sostenida en la matrícula de carreras de pedagogía en Chile plantea un desafío estratégico para las políticas públicas en educación. Comprender los factores que influyen en la decisión de ingresar a estas carreras resulta crucial para diseñar mecanismos de atracción y retención del talento docente.

Este trabajo desarrolla un modelo predictivo basado en técnicas de Machine Learning para anticipar si un estudiante de primer año optará por pedagogía o no, en función de variables académicas y demográficas. El enfoque permite aportar evidencia para **la toma de decisiones institucionales del MINEDUC**, tales como la **focalización territorial de esfuerzos, redistribución de recursos y diseño de estrategias de promoción vocacional docente.**

---

## 2. Objetivo del modelo

El objetivo general es construir un modelo de clasificación supervisada que, a partir de datos de matrícula de educación superior, prediga si una persona estudiará una carrera de pedagogía.

Los objetivos específicos son:
- Detectar combinaciones de variables personales y académicas asociadas a la elección de pedagogía.
- Comparar el rendimiento de distintos modelos de clasificación.
- Evaluar su aplicabilidad como instrumento para la planificación y focalización de políticas públicas en educación.

---

## 3. Descripción del Conjunto de Datos

### Fuente de los datos:
Los datos fueron obtenidos del portal oficial de Datos Abiertos del Ministerio de Educación de Chile:  
👉 [https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/](https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/)

### Estructura y filtrado:
- Se utilizaron los registros de matrícula de educación superior de los años **2023 y 2024**, post pandemia.
- Se filtraron exclusivamente las matrículas correspondientes a **primer año**, por ser el momento clave de elección vocacional.
- Dada la magnitud del dataset original (~900MB), se aplicó un muestreo aleatorio para obtener una versión manejable pero representativa.

---

## 4. Distribución y Preprocesamiento de los Datos

### Análisis exploratorio (`03analisis.ipynb`):
- Se revisaron tipos de datos, valores únicos y proporción de nulos.
- Se analizaron distribuciones de variables como: `rango_edad`, `jornada`, `modalidad`, `forma_ingreso`, etc.
- Se generaron boxplots y una matriz de correlación para explorar relaciones numéricas.
- Se identificaron y contabilizaron las carreras asociadas a pedagogía.

### Preprocesamiento (`04preprocesamiento.ipynb`):
- **Duplicados eliminados:** más de 1 millón de registros redundantes fueron eliminados.
- **Nulos eliminados:** se eliminaron 20 filas con datos faltantes.
- **Codificación:** uso de `LabelEncoder` para transformar variables categóricas en numéricas.
- **Normalización:** aplicación de `MinMaxScaler` sobre `valor_matricula` y `valor_arancel`.
- Se creó una nueva variable binaria `es_pedagogia` a partir del nombre de la carrera (`1` si contiene "PEDAGOGÍA").

---

## 5. Métodos y Modelos Utilizados

### Modelos entrenados (`05entrenamiento.ipynb`):
- **Árbol de Decisión (`DecisionTreeClassifier`)**: modelo interpretable, útil para identificar reglas lógicas.
- **Random Forest (`RandomForestClassifier`)**: modelo robusto, con mejor capacidad predictiva y menor sobreajuste.

### Hiperparámetros principales:
- Árbol de Decisión: `max_depth=4`
- Random Forest: `n_estimators=100`, `max_depth=10`, `random_state=42`

### División de datos:
- Se utilizó `train_test_split` con 80% de datos para entrenamiento y 20% para prueba.
- Se aplicó **estratificación por clase** (`es_pedagogia`) para asegurar representación balanceada.

---

## 6. Evaluación del Rendimiento del Modelo

Los modelos fueron evaluados en el notebook `06validacion.ipynb`.

### Métricas utilizadas:
1. **Accuracy:** proporción de predicciones correctas.
2. **Precision:** minimiza falsos positivos (importante para decisiones institucionales).
3. **Recall (Sensibilidad):** evita falsos negativos (relevante si se quiere asegurar inclusión de todos los casos potenciales).
4. **F1-score:** media armónica entre precisión y recall.
5. **Matriz de confusión:** para observar aciertos y errores por clase.

### Resultados comparativos:

| Modelo              | Accuracy | Precision (Pedagogía) | Recall (Pedagogía) | F1-score |
|---------------------|----------|------------------------|---------------------|----------|
| Árbol de Decisión   | 95.8%    | 0.76                   | 0.75                | 0.76     |
| Random Forest       | **96.9%**| **0.85**               | **0.77**            | **0.81** |

**Conclusión:** Random Forest se comporta mejor en la clase minoritaria (Pedagogía), con menor sobreajuste y mayor capacidad de generalización.

---

## 7. Interpretación de Resultados

- El modelo es capaz de identificar con alta precisión a los estudiantes con mayor propensión a estudiar pedagogía.
- Variables clave: `tipo_inst_1`, `region_sede`, `forma_ingreso`, `jornada`, y `nivel_carrera_1`.
- El árbol de decisión permite extraer reglas interpretables; el bosque aleatorio mejora la cobertura en los casos más difíciles.
- La matriz de confusión evidencia una buena proporción de verdaderos positivos sin exceso de falsos negativos.

---

## 8. Prueba con ejemplos reales (`07prueba.ipynb`)

Se evaluó el modelo final (Random Forest) con **tres perfiles sintéticos representativos** de estudiantes. Las predicciones fueron coherentes con las combinaciones de variables, mostrando:

- Sensibilidad al tipo de institución y forma de ingreso.
- Mayor probabilidad de predicción positiva cuando hay acreditación, jornada diurna y modalidad presencial.

Esto valida la aplicabilidad práctica del modelo en contextos institucionales reales.

---

## 9. Conclusiones

- El modelo propuesto puede ser una **herramienta efectiva para apoyar la planificación educativa del MINEDUC**, orientada a fortalecer la formación docente.
- Permite anticipar vocaciones pedagógicas desde datos disponibles, sin necesidad de instrumentos adicionales.
- Ofrece una base empírica para estrategias como becas, programas de atracción y difusión vocacional.
- Se recomienda su aplicación en **regiones con baja participación en pedagogía**, y su ampliación con modelos avanzados como `XGBoost` o `CatBoost`.

---

## 10. Referencias

- Ministerio de Educación de Chile. (2024). *Matrícula en Educación Superior*. Datos Abiertos.  
  https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

- Scikit-Learn Documentation. (2024). *User Guide*.  
  https://scikit-learn.org/stable/user_guide.html